In [1]:
import tensorflow as tf
import numpy as np
from ptr_decoder import pointer_decoder
from pointer_network import PointerNetwork
from data_generator import *
%load_ext autoreload
%autoreload 2

In [2]:
input_dim = 1
batch_size = 32
seq_len = 10
hidden_dim = 128
lr = 0.01
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
pointer_net = PointerNetwork(hidden_dim, lr, 10, seq_len, batch_size=batch_size, input_dim=1)

(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd_1:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd_2:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd_3:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd_4:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/decoder_1/transpose:0", shape=(32, 10, 1), dtype=float32)
Tensor("pointer_net/decoder_1/GatherNd_5:0", shape=(32, 1), dtype=float32)
(32, 10)
Tensor("pointer_net/d

In [4]:
opt = pointer_net.train_op
loss = pointer_net.loss
test_loss = pointer_net.test_loss
pred = pointer_net.predictions
decoder_outputs = pointer_net.decoder_outputs
sess.run(tf.initialize_all_variables())

In [5]:
# data = DataGenerator()
for i in range(0, 100000):
    _encoder_inps, _targets, _decoder_inps = sorting_generator(10, 32)
    feed_dict = pointer_net.feed_dict(decoder_inpt_data=_decoder_inps, 
                                      encoder_inpt_data=_encoder_inps, target_data=_targets)
    _test_loss, _pred = sess.run([test_loss, pred], feed_dict=feed_dict)
    _encoder_inps, _targets, _decoder_inps = sorting_generator(10, 32)
    #_encoder_inps, _decoder_inps, _targets = data.next_batch(2, 8)
    feed_dict = pointer_net.feed_dict(decoder_inpt_data=_decoder_inps, 
                                      encoder_inpt_data=_encoder_inps, target_data=_targets)

    _loss, _, _vs = sess.run([loss, opt, decoder_outputs], feed_dict=feed_dict)

    if i%100 == 0:   
        sample = np.random.randint(0, high=batch_size)
        print np.argmax(np.array(_targets)[:, sample, :], axis=1)
        print np.argmax(np.array(_vs)[:, sample, :], axis=1)
        print np.sum(np.all(np.argmax(np.array(_vs), axis=2)==np.argmax(np.array(_targets), axis=2), 
                     axis=0))
        print "Loss", _loss
        print "Test loss", _test_loss

[2 8 4 6 0 7 9 3 1 5]
[2 2 2 2 2 2 2 2 0 0]
0
Loss 18.6661
Test loss 19.2685
[4 2 3 5 1 9 8 6 7 0]
[4 4 4 4 4 0 0 0 0 0]
0
Loss 3.16893
Test loss 3.17286
[4 8 7 0 5 1 6 3 9 2]
[4 4 4 4 7 2 2 2 2 2]
0
Loss 2.85749
Test loss 2.85726
[6 3 8 5 4 1 0 9 7 2]
[6 6 3 8 5 9 9 2 2 2]
0
Loss 2.41304
Test loss 2.44281
[7 5 1 8 3 9 0 2 4 6]
[7 5 8 8 3 3 9 4 6 6]
0
Loss 2.03733
Test loss 2.04552
[8 6 3 4 0 9 5 1 2 7]
[8 3 3 3 4 5 5 1 2 7]
0
Loss 1.86365
Test loss 1.86016
[8 3 7 6 1 4 2 0 5 9]
[8 8 7 6 4 4 2 5 5 9]
0
Loss 1.78257
Test loss 1.79844
[9 8 3 0 6 2 5 7 4 1]
[9 8 3 3 6 6 5 7 1 1]
0
Loss 1.75401
Test loss 1.70054
[5 8 9 7 3 2 6 1 4 0]
[5 8 9 7 3 6 6 1 0 0]
0
Loss 1.79817
Test loss 1.63774
[8 0 9 6 4 5 7 1 2 3]
[8 8 6 6 6 4 7 1 2 3]
1
Loss 1.69207
Test loss 1.59024
[4 1 0 2 8 9 5 7 6 3]
[4 1 1 0 2 9 9 7 3 3]
2
Loss 1.55057
Test loss 1.50137
[4 3 1 9 5 7 0 2 8 6]
[3 1 1 9 5 7 7 6 6 6]
2
Loss 1.54345
Test loss 1.41872
[6 7 4 5 8 2 1 9 3 0]
[6 7 7 5 2 2 1 9 3 0]
5
Loss 1.39651
Test loss 1.4491


KeyboardInterrupt: 